This file is to find the similarity score of actual image and live captured images. And also it contains the different models to (experimented) to find the good model for calssifying real /fake

In [ ]:
from insightface.app import FaceAnalysis
import numpy as np
import cv2

app = FaceAnalysis(name='buffalo_l', providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
app.prepare(ctx_id=0, det_size=(224, 224))

def face_similarity(img1_path, img2_path):
    img1 = cv2.imread(img1_path)
    img2 = cv2.imread(img2_path)
    faces1 = app.get(img1)
    faces2 = app.get(img2)
    if not faces1 or not faces2:
        return 0.0
    emb1 = faces1[0]['embedding']
    emb2 = faces2[0]['embedding']
    sim = np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
    return sim

/usr/local/lib/python3.12/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:123: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (224, 224)


In [ ]:
img1_path="/content/1.jpeg"
img2_path="/content/2.jpeg"

In [ ]:
face_similarity(img1_path,img2_path)

np.float32(0.6651948)

In [ ]:
import torch
import clip
from PIL import Image
import numpy as np

In [ ]:


device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

def image_similarity(img1, img2):
    img1, img2 = preprocess(Image.open(img1)).unsqueeze(0).to(device), preprocess(Image.open(img2)).unsqueeze(0).to(device)
    with torch.no_grad():
        emb1, emb2 = model.encode_image(img1), model.encode_image(img2)
        emb1, emb2 = emb1 / emb1.norm(dim=-1, keepdim=True), emb2 / emb2.norm(dim=-1, keepdim=True)
    sim = (emb1 @ emb2.T).item()
    return sim


AttributeError: module 'clip' has no attribute 'load'

In [ ]:
image_similarity(img1_path,img2_path)

NameError: name 'image_similarity' is not defined

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import torch.nn.functional as F
from torchvision.models import vit_b_16

model = vit_b_16(weights="IMAGENET1K_V1").eval()

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,)*3, std=(0.5,)*3)
])

def get_features(img_path):
    img = transform(Image.open(img_path).convert("RGB")).unsqueeze(0)
    with torch.no_grad():
        return model(img).squeeze()

def similarity(img1, img2):
    f1, f2 = get_features(img1), get_features(img2)
    sim = F.cosine_similarity(f1, f2, dim=0).item()
    return sim


Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth


100%|██████████| 330M/330M [00:05<00:00, 66.6MB/s]


In [ ]:
similarity(img1_path,img2_path)

0.09921249747276306

In [ ]:
!pip uninstall -y clip
!pip install git+https://github.com/openai/CLIP.git


Found existing installation: clip 0.2.0
Uninstalling clip-0.2.0:
  Successfully uninstalled clip-0.2.0
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-7vp87bgb
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-7vp87bgb
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=4d19d66a905c77d823c40b0030359cbc822744df96288feffe5577e97ed62ebb
  Stored in directory: /tmp/pip-ephem-wheel-cache-8im11t7z/wheels/35/3e/df/3d24cbfb3b6a06f17a2bfd7d1138900d4365d9028aa8f6e92f
Successfully built clip


In [ ]:
import torch
import open_clip
from PIL import Image
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='openai', device=device)

def image_similarity(img1, img2):
    img1 = preprocess(Image.open(img1)).unsqueeze(0).to(device)
    img2 = preprocess(Image.open(img2)).unsqueeze(0).to(device)
    with torch.no_grad():
        emb1 = model.encode_image(img1)
        emb2 = model.encode_image(img2)
        emb1 /= emb1.norm(dim=-1, keepdim=True)
        emb2 /= emb2.norm(dim=-1, keepdim=True)
    return (emb1 @ emb2.T).item()

# Example
print("Similarity:", image_similarity(img1_path,img2_path))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


open_clip_model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/open_clip/factory.py:450: UserWarning: QuickGELU mismatch between final model config (quick_gelu=False) and pretrained tag 'openai' (quick_gelu=True).
  warnings.warn(


Similarity: 0.7617260217666626


In [ ]:
import torch
import clip
from PIL import Image
import pandas as pd
from tqdm import tqdm
import numpy as np

In [ ]:
import torch
import open_clip
from PIL import Image
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='openai', device=device)


# -------------------
# Load CLIP model
# -------------------
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model, preprocess = clip.load("ViT-B/32", device=device)

# -------------------
# Helper function
# -------------------
def get_similarity(img1_path, img2_path):
    try:
        img1 = preprocess(Image.open(img1_path).convert("RGB")).unsqueeze(0).to(device)
        img2 = preprocess(Image.open(img2_path).convert("RGB")).unsqueeze(0).to(device)
    except Exception as e:
        print(f"❌ Error reading image pair: {img1_path}, {img2_path}")
        return None

    with torch.no_grad():
        emb1 = model.encode_image(img1)
        emb2 = model.encode_image(img2)
        emb1 /= emb1.norm(dim=-1, keepdim=True)
        emb2 /= emb2.norm(dim=-1, keepdim=True)
        sim = (emb1 @ emb2.T).item()
    return sim

# -------------------
# Load CSV
# -------------------
csv_path = "/content/identity_pairs_50_val.csv"   # change if needed
df = pd.read_csv(csv_path)

# -------------------
# Compute similarities
# -------------------
similarities = []
labels = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    sim = get_similarity(row["img1"], row["img2"])
    if sim is not None:
        similarities.append(sim)
        labels.append(row["label"])

df["similarity"] = similarities

# -------------------
# Compute averages
# -------------------
same_person = df[df["label"] == 1]["similarity"]
diff_person = df[df["label"] == 0]["similarity"]

print(f"✅ Total pairs processed: {len(df)}")
print(f"Average similarity (same person): {same_person.mean():.4f}")
print(f"Average similarity (different persons): {diff_person.mean():.4f}")

# Save result CSV
df.to_csv("identity_val_similarity_results.csv", index=False)
print("✅ Saved results to identity_val_similarity_results.csv")


/usr/local/lib/python3.12/dist-packages/open_clip/factory.py:450: UserWarning: QuickGELU mismatch between final model config (quick_gelu=False) and pretrained tag 'openai' (quick_gelu=True).
  warnings.warn(
100%|██████████| 2000/2000 [18:08<00:00,  1.84it/s]

✅ Total pairs processed: 2000
Average similarity (same person): 0.8311
Average similarity (different persons): 0.7264
✅ Saved results to identity_val_similarity_results.csv


insightface-based similarity function

In [ ]:
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from insightface.app import FaceAnalysis

# -------------------
# Initialize InsightFace model
# -------------------
app = FaceAnalysis(name='buffalo_l', providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
app.prepare(ctx_id=0, det_size=(224, 224))

# -------------------
# Helper: compute similarity between two face images
# -------------------
def face_similarity(img1_path, img2_path):
    img1 = cv2.imread(img1_path)
    img2 = cv2.imread(img2_path)

    if img1 is None or img2 is None:
        print(f"⚠️ Could not read one of the images: {img1_path}, {img2_path}")
        return None

    faces1 = app.get(img1)
    faces2 = app.get(img2)

    if len(faces1) == 0 or len(faces2) == 0:
        print(f"⚠️ No face detected in: {img1_path} or {img2_path}")
        return None

    emb1 = faces1[0]['embedding']
    emb2 = faces2[0]['embedding']

    # Cosine similarity
    sim = np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
    return float(sim)

# -------------------
# Load CSV file
# -------------------
csv_path = "/content/identity_pairs_50_val.csv"  # change to your path
df = pd.read_csv(csv_path)

# Expecting columns: img1, img2, label
assert all(col in df.columns for col in ['img1', 'img2', 'label']), "CSV must have columns: img1,img2,label"

# -------------------
# Compute similarities for all pairs
# -------------------
similarities = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    sim = face_similarity(row['img1'], row['img2'])
    similarities.append(sim if sim is not None else 0.0)

df["similarity"] = similarities

# -------------------
# Compute summary statistics
# -------------------
same_person = df[df["label"] == 1]["similarity"]
diff_person = df[df["label"] == 0]["similarity"]

print("\n✅ Results Summary:")
print(f"Total pairs processed: {len(df)}")
print(f"Average similarity (same person): {same_person.mean():.4f}")
print(f"Average similarity (different persons): {diff_person.mean():.4f}")

# -------------------
# Save results
# -------------------
df.to_csv("identity_similarity_results.csv", index=False)
print("✅ Saved results to identity_similarity_results.csv")


/usr/local/lib/python3.12/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:123: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (224, 224)


 12%|█▏        | 230/2000 [05:36<34:11,  1.16s/it]

⚠️ No face detected in: /content/content/processed_faces/duplication/n000314/0478_01.jpg or /content/content/processed_faces/duplication/n000314/0127_02.jpg


 36%|███▋      | 729/2000 [17:34<25:49,  1.22s/it]

...

⚠️ No face detected in: /content/content/processed_faces/duplication/n000277/0108_01.jpg or /content/content/processed_faces/duplication/n000055/0308_01.jpg


 82%|████████▏ | 1637/2000 [39:21<07:30,  1.24s/it]

⚠️ No face detected in: /content/content/processed_faces/duplication/n000314/0478_01.jpg or /content/content/processed_faces/duplication/n000314/0388_02.jpg


100%|██████████| 2000/2000 [48:01<00:00,  1.44s/it]


✅ Results Summary:
Total pairs processed: 2000
Average similarity (same person): 0.5068
Average similarity (different persons): 0.0039
✅ Saved results to identity_similarity_results.csv


deepfake

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as T
from PIL import Image
import cv2
import numpy as np
from insightface.app import FaceAnalysis

# -------------------------------
# 1️⃣ Initialize InsightFace for face detection
# -------------------------------
app = FaceAnalysis(name='buffalo_l')
app.prepare(ctx_id=0, det_size=(224, 224))

# -------------------------------
# 2️⃣ Load pretrained XceptionNet for deepfake detection
# -------------------------------
# You can download a pretrained checkpoint from FaceForensics++
# For demonstration, we will use timm's xception model backbone (replace with your checkpoint)
import timm

class DeepfakeDetector(nn.Module):
    def __init__(self, backbone_name="xception", pretrained=True):
        super(DeepfakeDetector, self).__init__()
        self.backbone = timm.create_model(backbone_name, pretrained=pretrained, num_classes=2)

    def forward(self, x):
        return self.backbone(x)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = DeepfakeDetector(pretrained=True).to(device)
model.eval()

# -------------------------------
# 3️⃣ Transform function
# -------------------------------
transform = T.Compose([
    T.ToPILImage(),
    T.Resize((299, 299)),  # Xception expects 299x299
    T.ToTensor(),
    T.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# -------------------------------
# 4️⃣ Face crop + deepfake detection
# -------------------------------
def detect_deepfake(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"⚠️ Image not found: {image_path}")
        return None

    faces = app.get(img)
    if not faces:
        print(f"⚠️ No face detected in {image_path}")
        return None

    # Take largest face
    face = max(faces, key=lambda x: (x.bbox[2]-x.bbox[0])*(x.bbox[3]-x.bbox[1]))

    # Manually crop using bounding box
    x1, y1, x2, y2 = map(int, face.bbox)
    face_crop = img[y1:y2, x1:x2]

    if face_crop.size == 0:
        print(f"⚠️ Empty crop for {image_path}")
        return None

    # Transform
    inp = transform(face_crop).unsqueeze(0).to(device)

    # Inference
    with torch.no_grad():
        logits = model(inp)
        probs = nn.Softmax(dim=1)(logits)
        real_prob = probs[0,0].item()
        fake_prob = probs[0,1].item()

    return {"real_prob": real_prob, "fake_prob": fake_prob}
# -------------------------------
# 5️⃣ Example usage
# -------------------------------
image_path = "/content/f5.jpeg"
result = detect_deepfake(image_path)
if result:
    print(f"Real: {result['real_prob']:.3f}, Fake: {result['fake_prob']:.3f}")

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (224, 224)
Real: 0.512,

Xception pretrained on FaceForensics++,

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as T
import cv2
import numpy as np
from PIL import Image
from insightface.app import FaceAnalysis

# ================================
# 1️⃣ Initialize InsightFace for face detection
# ================================
app = FaceAnalysis(name='buffalo_l')
app.prepare(ctx_id=0, det_size=(224, 224))

# ================================
# 2️⃣ Define Xception model for deepfake detection
# ================================
import timm

class XceptionDeepfake(nn.Module):
    def __init__(self, pretrained=True):
        super(XceptionDeepfake, self).__init__()
        # num_classes=2 for real/fake
        self.model = timm.create_model('xception', pretrained=pretrained, num_classes=2)

    def forward(self, x):
        return self.model(x)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = XceptionDeepfake(pretrained=True).to(device)
model.eval()

# ================================
# 3️⃣ Transform for face crops
# ================================
transform = T.Compose([
    T.ToPILImage(),
    T.Resize((299, 299)),  # Xception expects 299x299
    T.ToTensor(),
    T.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# ================================
# 4️⃣ Deepfake detection function
# ================================
def detect_deepfake(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"⚠️ Image not found: {image_path}")
        return None

    faces = app.get(img)
    if not faces:
        print(f"⚠️ No face detected in {image_path}")
        return None

    # Take largest face
    face = max(faces, key=lambda x: (x.bbox[2]-x.bbox[0])*(x.bbox[3]-x.bbox[1]))

    # Crop manually using bounding box
    x1, y1, x2, y2 = map(int, face.bbox)
    face_crop = img[y1:y2, x1:x2]

    if face_crop.size == 0:
        print(f"⚠️ Empty crop for {image_path}")
        return None

    inp = transform(face_crop).unsqueeze(0).to(device)

    with torch.no_grad():
        logits = model(inp)
        probs = nn.Softmax(dim=1)(logits)
        real_prob = probs[0,0].item()
        fake_prob = probs[0,1].item()

    return {"real_prob": real_prob, "fake_prob": fake_prob}

# ================================
# 5️⃣ Example usage
# ================================
image_path = "/content/f5.jpeg"
result = detect_deepfake(image_path)
if result:
    print(f"Real: {result['real_prob']:.3f}, Fake: {result['fake_prob']:.3f}")

# ================================
# 6️⃣ Optional: Compute face similarity if real
# ================================
def face_similarity(img1_path, img2_path):
    img1 = cv2.imread(img1_path)
    img2 = cv2.imread(img2_path)

    faces1 = app.get(img1)
    faces2 = app.get(img2)

    if not faces1 or not faces2:
        return 0.0

    emb1 = faces1[0]['embedding']
    emb2 = faces2[0]['embedding']

    sim = np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
    return sim


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (224, 224)
Real: 0.495,

In [ ]:
result = detect_deepfake("/content/live_face.jpg")
if result and result['real_prob'] > 0.7:
    sim_score = face_similarity("/content/live_face.jpg", "/content/original_face.jpg")
    print("Similarity score:", sim_score)
else:
    print("Face is likely fake, rejecting similarity check.")


In [ ]:
import torch

checkpoint_path = "/content/xception-b5690688.pth"
checkpoint = torch.load(checkpoint_path)

# Remove 'model.' prefix if exists
new_state_dict = {}
for k, v in checkpoint.items():
    if k.startswith("model."):
        new_key = k.replace("model.", "")
    else:
        new_key = k
    new_state_dict[new_key] = v

# Load into model
model = XceptionDeepfake()  # your class
model.load_state_dict(new_state_dict, strict=False)

/usr/local/lib/python3.12/dist-packages/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(


_IncompatibleKeys(missing_keys=['model.conv1.weight', 'model.bn1.weight', 'model.bn1.bias', 'model.bn1.running_mean', 'model.bn1.running_var', 'model.conv2.weight', 'model.bn2.weight', 'model.bn2.bias', 'model.bn2.running_mean', 'model.bn2.running_var', 'model.block1.skip.weight', 'model.block1.skipbn.weight', 'model.block1.skipbn.bias', 'model.block1.skipbn.running_mean', 'model.block1.skipbn.running_var', 'model.block1.rep.0.conv1.weight', 'model.block1.rep.0.pointwise.weight', 'model.block1.rep.1.weight', 'model.block1.rep.1.bias', 'model.block1.rep.1.running_mean', 'model.block1.rep.1.running_var', 'model.block1.rep.3.conv1.weight', 'model.block1.rep.3.pointwise.weight', 'model.block1.rep.4.weight', 'model.block1.rep.4.bias', 'model.block1.rep.4.running_mean', 'model.block1.rep.4.running_var', 'model.block2.skip.weight', 'model.block2.skipbn.weight', 'model.block2.skipbn.bias', 'model.block2.skipbn.running_mean', 'model.block2.skipbn.running_var', 'model.block2.rep.1.conv1.weight',

In [ ]:
import timm
import torch
import torch.nn as nn

# Create Xception backbone (num_classes=2 for real/fake)
model = timm.create_model('xception', pretrained=False, num_classes=2)

# Load checkpoint directly
state_dict = torch.load("/content/xception-b5690688.pth")

# Remove the final classifier weights from the state dictionary as they cause size mismatch
# The keys might vary slightly depending on the timm version and how the checkpoint was saved.
# Based on the error message, the keys seem to be 'fc.weight' and 'fc.bias'.
if 'fc.weight' in state_dict:
    del state_dict['fc.weight']
if 'fc.bias' in state_dict:
    del state_dict['fc.bias']

# Load into model, ignoring any remaining mismatched keys (e.g., in intermediate layers if any)
model.load_state_dict(state_dict, strict=False)
model.eval()

RuntimeError: Error(s) in loading state_dict for Xception:
	size mismatch for block1.rep.0.pointwise.weight: copying a param with shape torch.Size([128, 64]) from checkpoint, the shape in current model is torch.Size([128, 64, 1, 1]).
	size mismatch for block1.rep.3.pointwise.weight: copying a param with shape torch.Size([128, 128]) from checkpoint, the shape in current model is torch.Size([128, 128, 1, 1]).
	size mismatch for block2.rep.1.pointwise.weight: copying a param with shape torch.Size([256, 128]) from checkpoint, the shape in current model is torch.Size([256, 128, 1, 1]).
	size mismatch for block2.rep.4.pointwise.weight: copying a param with shape torch.Size([256, 256]) from checkpoint, the shape in current model is torch.Size([256, 256, 1, 1]).
	size mismatch for block3.rep.1.pointwise.weight: copying a param with shape torch.Size([728, 256]) from checkpoint, the shape in current model is torch.Size([728, 256, 1, 1]).
	size mismatch for block3.rep.4.pointwise.weight: copying 

FaceForensics++

In [ ]:
# import torch
# import timm
# from torch import nn

# class XceptionDeepfake(nn.Module):
#     def __init__(self, pretrained_path=None):
#         super(XceptionDeepfake, self).__init__()
#         # num_classes=2 for real/fake
#         self.model = timm.create_model('xception', pretrained=False, num_classes=2)
#         if pretrained_path:
#             # Load the state dictionary into the self.model (timm Xception model)
#             self.model.load_state_dict(torch.load(pretrained_path))

#     def forward(self, x):
#         return self.model(x)

# # Initialize the model
# model = XceptionDeepfake(pretrained_path='/content/xception-b5690688.pth')
# model.eval()

/usr/local/lib/python3.12/dist-packages/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(


RuntimeError: Error(s) in loading state_dict for Xception:
	size mismatch for block1.rep.0.pointwise.weight: copying a param with shape torch.Size([128, 64]) from checkpoint, the shape in current model is torch.Size([128, 64, 1, 1]).
	size mismatch for block1.rep.3.pointwise.weight: copying a param with shape torch.Size([128, 128]) from checkpoint, the shape in current model is torch.Size([128, 128, 1, 1]).
	size mismatch for block2.rep.1.pointwise.weight: copying a param with shape torch.Size([256, 128]) from checkpoint, the shape in current model is torch.Size([256, 128, 1, 1]).
	size mismatch for block2.rep.4.pointwise.weight: copying a param with shape torch.Size([256, 256]) from checkpoint, the shape in current model is torch.Size([256, 256, 1, 1]).
	size mismatch for block3.rep.1.pointwise.weight: copying a param with shape torch.Size([728, 256]) from checkpoint, the shape in current model is torch.Size([728, 256, 1, 1]).
	size mismatch for block3.rep.4.pointwise.weight: copying 

In [ ]:
import torchvision.transforms as T
from PIL import Image
import cv2

transform = T.Compose([
    T.Resize((299, 299)),
    T.ToTensor(),
    T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

def preprocess_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Image not found: {image_path}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(img)
    return transform(pil_img).unsqueeze(0)


In [ ]:
import torch.nn.functional as F

def detect_deepfake(image_path):
    input_tensor = preprocess_image(image_path)
    with torch.no_grad():
        logits = model(input_tensor)
        probs = F.softmax(logits, dim=1)
        real_prob = probs[0, 0].item()
        fake_prob = probs[0, 1].item()
    return {"real_prob": real_prob, "fake_prob": fake_prob}

# Example usage
image_path = '/content/r2.jpeg'
result = detect_deepfake(image_path)
print(f"Real: {result['real_prob']:.3f}, Fake: {result['fake_prob']:.3f}")


Real: 0.423, Fake: 0.577


In [ ]:
from insightface.app import FaceAnalysis

# Initialize InsightFace for face detection
app = FaceAnalysis(name='buffalo_l')
app.prepare(ctx_id=0, det_size=(224, 224))

def detect_face(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Image not found: {image_path}")
    faces = app.get(img)
    if not faces:
        raise ValueError(f"No face detected in {image_path}")
    # Assuming the first detected face is the one of interest
    face = faces[0]
    x1, y1, x2, y2 = map(int, face.bbox)
    face_crop = img[y1:y2, x1:x2]
    return face_crop

# Example usage
image_path = '/content/r1.jpeg'
face_crop = detect_face(image_path)
# Now you can pass face_crop to the detect_deepfake function


/usr/local/lib/python3.12/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:123: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (224, 224)


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as T
from PIL import Image
import cv2
import numpy as np
from insightface.app import FaceAnalysis
from tqdm import tqdm
import os

# ================================
# 1️⃣ Initialize InsightFace for face detection
# ================================
app = FaceAnalysis(name='buffalo_l')
app.prepare(ctx_id=0, det_size=(224, 224))

# ================================
# 2️⃣ Load Xception pretrained on FaceForensics++
# =================================
import timm

class DeepfakeDetector(nn.Module):
    def __init__(self, backbone_name="xception", pretrained_ckpt=None):
        super(DeepfakeDetector, self).__init__()
        # Xception backbone with 2 output classes (real/fake)
        self.model = timm.create_model(backbone_name, pretrained=False, num_classes=2)

        # Load pretrained FaceForensics++ checkpoint if provided
        if pretrained_ckpt:
            state_dict = torch.load(pretrained_ckpt, map_location='cpu')
            # Handle key prefix mismatch if necessary
            new_state_dict = {}
            for k, v in state_dict.items():
                # Remove 'model.' prefix if it exists
                if k.startswith("model."):
                    new_key = k.replace("model.", "")
                else:
                    new_key = k
                new_state_dict[new_key] = v

            # Remove the final classifier weights from the state dictionary as they cause size mismatch
            # The keys might vary slightly depending on the timm version and how the checkpoint was saved.
            # Based on the error message, the keys seem to be 'fc.weight' and 'fc.bias'.
            keys_to_remove = ['fc.weight', 'fc.bias']
            for key in keys_to_remove:
                if key in new_state_dict:
                    del new_state_dict[key]

            # Load into model, ignoring any remaining mismatched keys (e.g., in intermediate layers if any)
            self.model.load_state_dict(new_state_dict, strict=False)

    def forward(self, x):
        return self.model(x)

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize model
checkpoint_path = "/content/xception-b5690688.pth"  # path to FaceForensics++ pretrained checkpoint
model = DeepfakeDetector(pretrained_ckpt=checkpoint_path).to(device)
model.eval()

# ================================
# 3️⃣ Transform function for Xception
# ================================
transform = T.Compose([
    T.ToPILImage(),
    T.Resize((299, 299)),  # Xception expects 299x299
    T.ToTensor(),
    T.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# ================================
# 4️⃣ Function to detect deepfake in a single image
# ================================
def detect_deepfake(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"⚠️ Image not found: {image_path}")
        return None

    faces = app.get(img)
    if not faces:
        print(f"⚠️ No face detected in {image_path}")
        return None

    # Take the largest face
    face = max(faces, key=lambda x: (x.bbox[2]-x.bbox[0])*(x.bbox[3]-x.bbox[1]))
    x1, y1, x2, y2 = map(int, face.bbox)
    face_crop = img[y1:y2, x1:x2]

    if face_crop.size == 0:
        print(f"⚠️ Empty crop for {image_path}")
        return None

    # Transform and move to device
    inp = transform(face_crop).unsqueeze(0).to(device)

    # Inference
    with torch.no_grad():
        logits = model(inp)
        probs = nn.Softmax(dim=1)(logits)
        real_prob = probs[0, 0].item()
        fake_prob = probs[0, 1].item()

    return {"real_prob": real_prob, "fake_prob": fake_prob}

# ================================
# 5️⃣ Example usage for a single image
# ================================
image_path = "/content/r2.jpeg"
result = detect_deepfake(image_path)
if result:
    print(f"Real probability: {result['real_prob']:.3f}, Fake probability: {result['fake_prob']:.3f}")

# ================================
# 6️⃣ Optional: Batch inference for folder
# ================================
# def detect_deepfake_folder(folder_path):
#     results = []
#     for img_name in tqdm(os.listdir(folder_path)):
#         if not img_name.lower().endswith(('.jpg', '.png', '.jpeg')):
#             continue
#         img_path = os.path.join(folder_path, img_name)
#         res = detect_deepfake(img_path)
#         if res:
#             res['image'] = img_name
#             results.append(res)
#     return results

# Example usage:
# folder_results = detect_deepfake_folder("/content/test_images")
# for r in folder_results:
#     print(r)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (224, 224)


RuntimeError: Error(s) in loading state_dict for Xception:
	size mismatch for block1.rep.0.pointwise.weight: copying a param with shape torch.Size([128, 64]) from checkpoint, the shape in current model is torch.Size([128, 64, 1, 1]).
	size mismatch for block1.rep.3.pointwise.weight: copying a param with shape torch.Size([128, 128]) from checkpoint, the shape in current model is torch.Size([128, 128, 1, 1]).
	size mismatch for block2.rep.1.pointwise.weight: copying a param with shape torch.Size([256, 128]) from checkpoint, the shape in current model is torch.Size([256, 128, 1, 1]).
	size mismatch for block2.rep.4.pointwise.weight: copying a param with shape torch.Size([256, 256]) from checkpoint, the shape in current model is torch.Size([256, 256, 1, 1]).
	size mismatch for block3.rep.1.pointwise.weight: copying a param with shape torch.Size([728, 256]) from checkpoint, the shape in current model is torch.Size([728, 256, 1, 1]).
	size mismatch for block3.rep.4.pointwise.weight: copying 

ValueError: Unrecognized model in ArissBandoss/deepfake-video-classifier. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: aimv2, aimv2_vision_model, albert, align, altclip, apertus, arcee, aria, aria_text, audio-spectrogram-transformer, autoformer, aya_vision, bamba, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, bitnet, blenderbot, blenderbot-small, blip, blip-2, blip_2_qformer, bloom, blt, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_text_model, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, cohere2, cohere2_vision, colpali, colqwen2, conditional_detr, convbert, convnext, convnextv2, cpmant, csm, ctrl, cvt, d_fine, dab-detr, dac, data2vec-audio, data2vec-text, data2vec-vision, dbrx, deberta, deberta-v2, decision_transformer, deepseek_v2, deepseek_v3, deepseek_vl, deepseek_vl_hybrid, deformable_detr, deit, depth_anythin

/usr/local/lib/python3.12/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(


Downloading: "https://github.com/ondyari/FaceForensics/zipball/master" to /root/.cache/torch/hub/master.zip


FileNotFoundError: [Errno 2] No such file or directory: '/root/.cache/torch/hub/ondyari_FaceForensics_master/hubconf.py'

In [ ]:
transform = T.Compose([
    T.ToPILImage(),
    T.Resize((299, 299)),  # Xception expects 299x299
    T.ToTensor(),
    T.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# ================================
# 4️⃣ Function to detect deepfake in a single image
# ================================
def detect_deepfake(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"⚠️ Image not found: {image_path}")
        return None

    faces = app.get(img)
    if not faces:
        print(f"⚠️ No face detected in {image_path}")
        return None

    # Take the largest face
    face = max(faces, key=lambda x: (x.bbox[2]-x.bbox[0])*(x.bbox[3]-x.bbox[1]))
    x1, y1, x2, y2 = map(int, face.bbox)
    face_crop = img[y1:y2, x1:x2]

    if face_crop.size == 0:
        print(f"⚠️ Empty crop for {image_path}")
        return None

    # Transform and move to device
    inp = transform(face_crop).unsqueeze(0).to(device)

    # Inference
    with torch.no_grad():
        logits = model(inp)
        probs = nn.Softmax(dim=1)(logits)
        real_prob = probs[0, 0].item()
        fake_prob = probs[0, 1].item()

    return {"real_prob": real_prob, "fake_prob": fake_prob}

# ================================
# 5️⃣ Example usage for a single image
# ================================
image_path = "/content/r2.jpeg"
result = detect_deepfake(image_path)
if result:
    print(f"Real probability: {result['real_prob']:.3f}, Fake probability: {result['fake_prob']:.3f}")


In [ ]:
'''working fine but results are not good'''

# import cv2
# import numpy as np
# import torch
# import torch.nn.functional as F
# from insightface.app import FaceAnalysis
# from torchvision import transforms
# from PIL import Image

# # -------------------------
# # 1️⃣ Initialize face detector
# # -------------------------
# app = FaceAnalysis(name='buffalo_l')
# app.prepare(ctx_id=0, det_size=(224, 224))

# # -------------------------
# # 2️⃣ Preprocessing
# # -------------------------
# transform = transforms.Compose([
#     transforms.Resize((299, 299)),  # Xception input
#     transforms.ToTensor(),
#     transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
# ])

# # -------------------------
# # 3️⃣ Deepfake detection function
# # -------------------------
# def detect_deepfake(image_path):
#     img = cv2.imread(image_path)
#     if img is None:
#         print(f"Image not found: {image_path}")
#         return None

#     faces = app.get(img)
#     if not faces:
#         print(f"No face detected: {image_path}")
#         return None

#     # Take largest face
#     face = max(faces, key=lambda x: (x.bbox[2]-x.bbox[0])*(x.bbox[3]-x.bbox[1]))
#     x1, y1, x2, y2 = map(int, face.bbox)
#     face_crop = img[y1:y2, x1:x2]
#     if face_crop.size == 0:
#         return None

#     # Transform and add batch dimension
#     face_pil = Image.fromarray(cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB))
#     inp = transform(face_pil).unsqueeze(0).to(device)

#     # Inference
#     with torch.no_grad():
#         logits = model(inp)
#         probs = F.softmax(logits, dim=1)
#         real_prob = probs[0,0].item()
#         fake_prob = probs[0,1].item()

#     return {"real_prob": real_prob, "fake_prob": fake_prob}

# # -------------------------
# # 4️⃣ Example usage
# # -------------------------
# image_path = "/content/r2.jpeg"
# result = detect_deepfake(image_path)
# if result:
#     print(f"Real: {result['real_prob']:.3f}, Fake: {result['fake_prob']:.3f}")


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (224, 224)
Real: 0.423,

In [ ]:
# !git clone https://github.com/DariusAf/MesoNet.git


Cloning into 'MesoNet'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 82 (delta 29), reused 22 (delta 22), pack-reused 47 (from 3)
Receiving objects: 100% (82/82), 494.53 KiB | 8.99 MiB/s, done.
Resolving deltas: 100% (32/32), done.


In [ ]:
# !ls MesoNet


classifiers.py	LICENSE  pipeline.py  test_images
example.py	NOTICE	 README.md    weights


In [ ]:
# !grep -R "class MesoInception4" MesoNet/


MesoNet/classifiers.py:class MesoInception4(Classifier):


In [ ]:
# from classifiers import MesoInception4
# import torch

# device = "cpu"  # Must run on CPU for this repo
# model = MesoInception4()

# # Load checkpoint
# checkpoint_path = "/content/MesoNet/weights/MesoInception_DF.h5"
# checkpoint = torch.load(checkpoint_path, map_location='cpu', weights_only=False)

# if 'model' in checkpoint:
#     model.model = checkpoint['model']
# else:
#     model.model = checkpoint

# # Inference
# from PIL import Image
# from torchvision import transforms

# preprocess = transforms.Compose([
#     transforms.Resize((256, 256)),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
# ])

# img = Image.open("/content/test_face.jpg").convert("RGB")
# inp = preprocess(img).unsqueeze(0)

# # Forward
# with torch.no_grad():
#     logits = model.forward(inp)  # uses functional forward
#     probs = torch.softmax(logits, dim=1)
#     print("Real:", probs[0,0].item(), "Fake:", probs[0,1].item())


UnpicklingError: invalid load key, 'H'.

In [ ]:
# import sys
# sys.path.append('/content/MesoNet')  # path to cloned repo

# from meso_inception4 import MesoInception4
# import torch

# # Load pretrained weights (download the .pth manually if needed)
# # Make sure 'meso_inception4_DF.pth' is in the correct path or provide the full path
# checkpoint_path = "/content/MesoNet/weights/meso_inception4_DF.pth" # Example path assuming weights are in a 'weights' subfolder
# model = MesoInception4()
# checkpoint = torch.load(checkpoint_path, map_location='cpu')
# model.load_state_dict(checkpoint)
# model.eval()
# model.to("cuda" if torch.cuda.is_available() else "cpu")

ModuleNotFoundError: No module named 'meso_inception4'

In [ ]:
# # -------------------------------
# # 1️⃣ Install requirements
# # -------------------------------
# # pip install insightface opencv-python torch torchvision tqdm

# import torch
# import torch.nn as nn
# import torchvision.transforms as T
# import cv2
# import numpy as np
# from insightface.app import FaceAnalysis
# from tqdm import tqdm

# # -------------------------------
# # 2️⃣ Initialize InsightFace for face detection
# # -------------------------------
# app = FaceAnalysis(name='buffalo_l')
# app.prepare(ctx_id=0, det_size=(224, 224))

# # -------------------------------
# # 3️⃣ Load pretrained MesoInception-4 (MesoNet)
# # -------------------------------
# # Using PyTorch Hub
# model = torch.hub.load('DariusAf/MesoNet', 'mesoinception4', pretrained=True)
# model.eval()
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model.to(device)

# # -------------------------------
# # 4️⃣ Define transforms for MesoNet
# # -------------------------------
# transform = T.Compose([
#     T.ToPILImage(),
#     T.Resize((256, 256)),  # MesoNet input size
#     T.ToTensor(),
#     T.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
# ])

# # -------------------------------
# # 5️⃣ Face crop + deepfake detection
# # -------------------------------
# def detect_face_and_fake(image_path):
#     """
#     Returns:
#         dict with keys:
#             - face_crop: cropped face image
#             - real_prob: probability image is real
#             - fake_prob: probability image is fake
#     """
#     img = cv2.imread(image_path)
#     if img is None:
#         print(f"⚠️ Image not found: {image_path}")
#         return None

#     # Detect faces
#     faces = app.get(img)
#     if not faces:
#         print(f"⚠️ No face detected in {image_path}")
#         return None

#     # Take largest face
#     face = max(faces, key=lambda x: (x.bbox[2]-x.bbox[0])*(x.bbox[3]-x.bbox[1]))
#     x1, y1, x2, y2 = map(int, face.bbox)
#     face_crop = img[y1:y2, x1:x2]

#     if face_crop.size == 0:
#         print(f"⚠️ Empty crop for {image_path}")
#         return None

#     # Transform and forward pass
#     inp = transform(face_crop).unsqueeze(0).to(device)
#     with torch.no_grad():
#         logits = model(inp)
#         probs = nn.Softmax(dim=1)(logits)
#         real_prob = probs[0,0].item()
#         fake_prob = probs[0,1].item()

#     return {
#         "face_crop": face_crop,
#         "real_prob": real_prob,
#         "fake_prob": fake_prob
#     }

# # -------------------------------
# # 6️⃣ Example usage
# # -------------------------------
# image_path = "example.jpg"  # Replace with your image
# result = detect_face_and_fake(image_path)

# if result:
#     print(f"Real probability: {result['real_prob']:.3f}")
#     print(f"Fake probability: {result['fake_prob']:.3f}")

#     # Optional: show cropped face
#     cv2.imshow("Face Crop", result["face_crop"])
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()


/usr/local/lib/python3.12/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:123: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (224, 224)


Using cache found in /root/.cache/torch/hub/DariusAf_MesoNet_master


FileNotFoundError: [Errno 2] No such file or directory: '/root/.cache/torch/hub/DariusAf_MesoNet_master/hubconf.py'

In [ ]:
# from transformers import AutoProcessor, AutoModelForImageClassification
# import torch
# from PIL import Image

# processor = AutoProcessor.from_pretrained("azminet/deepfake-image-detector")
# model = AutoModelForImageClassification.from_pretrained("azminet/deepfake-image-detector")
# model.eval()

# img = Image.open("example.jpg")
# inputs = processor(images=img, return_tensors="pt")
# with torch.no_grad():
#     logits = model(**inputs).logits
#     probs = torch.softmax(logits, dim=1)
#     print("Real:", probs[0][0].item(), "Fake:", probs[0][1].item())


OSError: azminet/deepfake-image-detector is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`

Using the prithivMLmods/Deep-Fake-Detector-v2-Model

In [ ]:
from transformers import AutoModelForImageClassification, AutoProcessor

model = AutoModelForImageClassification.from_pretrained("prithivMLmods/Deep-Fake-Detector-v2-Model")
processor = AutoProcessor.from_pretrained("prithivMLmods/Deep-Fake-Detector-v2-Model")


config.json:   0%|          | 0.00/738 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/351 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
from transformers import AutoModelForImageClassification, AutoProcessor

model = AutoModelForImageClassification.from_pretrained("prithivMLmods/Deep-Fake-Detector-v2-Model")
processor = AutoProcessor.from_pretrained("prithivMLmods/Deep-Fake-Detector-v2-Model")


In [ ]:
# import torch

# from PIL import Image

# img = Image.open("/content/r1.jpeg").convert("RGB")
# inputs = processor(images=img, return_tensors="pt")

# with torch.no_grad():
#     logits = model(**inputs).logits
#     predicted_class_idx = logits.argmax(-1).item()

# labels = ["Real", "Fake"]
# print(f"Prediction: {labels[predicted_class_idx]}")


Prediction: Fake


In [ ]:
# !file /content/MesoNet/weights/MesoInception_DF.h5

/content/MesoNet/weights/MesoInception_DF.h5: Hierarchical Data Format (version 5) data


In [ ]:
# !pip install  tensorflow opencv-python


In [ ]:
# # -------------------------------
# # 1️⃣ Install dependencies
# # -------------------------------


# # -------------------------------
# # 2️⃣ Imports
# # -------------------------------
# import cv2
# import numpy as np
# import tensorflow as tf # Import tensorflow
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import (
#     Input, Conv2D, BatchNormalization, AveragePooling2D,
#     Flatten, Dense, Dropout
# )
# from tensorflow.keras.preprocessing.image import img_to_array
# from insightface.app import FaceAnalysis
# from tqdm import tqdm

# # -------------------------------
# # 3️⃣ Define MesoInception4 architecture
# # -------------------------------
# def InceptionModule(x, filters):
#     tower_1 = Conv2D(filters, (1,1), padding='same', activation='relu')(x)
#     tower_2 = Conv2D(filters, (3,3), padding='same', activation='relu')(x)
#     tower_3 = Conv2D(filters, (5,5), padding='same', activation='relu')(x)
#     tower_4 = Conv2D(filters, (11,11), padding='same', activation='relu')(x)
#     output = tf.keras.layers.concatenate([tower_1, tower_2, tower_3, tower_4], axis=-1)
#     return output

# def MesoInception4():
#     inp = Input(shape=(256, 256, 3))
#     x = InceptionModule(inp, 8)
#     x = AveragePooling2D(pool_size=(2, 2))(x)
#     x = InceptionModule(x, 8)
#     x = AveragePooling2D(pool_size=(2, 2))(x)
#     x = Conv2D(16, (5, 5), padding='same', activation='relu')(x)
#     x = AveragePooling2D(pool_size=(4, 4))(x)
#     x = Flatten()(x)
#     x = Dropout(0.5)(x)
#     x = Dense(16, activation='relu')(x)
#     out = Dense(1, activation='sigmoid')(x)
#     model = Model(inp, out)
#     return model

# # -------------------------------
# # 4️⃣ Create model and load weights
# # -------------------------------
# checkpoint_path = "/content/MesoNet/weights/MesoInception_DF.h5"
# deepfake_model = MesoInception4()
# deepfake_model.load_weights(checkpoint_path)
# print("✅ Loaded MesoInception4 weights successfully!")

ValueError: Layer count mismatch when loading weights from file. Model expected 11 layers, found 22 saved layers.

In [ ]:
# import sys, os
# sys.path.append('/content/MesoNet')  # path where repo was cloned

# from classifiers import MesoInception4
# import tensorflow as tf
# import cv2
# import numpy as np
# from tensorflow.keras.preprocessing.image import img_to_array
# from insightface.app import FaceAnalysis

# # -------------------------------
# # 2️⃣ Load pretrained model
# # -------------------------------
# checkpoint_path = "/content/MesoNet/weights/MesoInception_DF.h5"

# # The wrapper class
# model_wrapper = MesoInception4()
# # The actual Keras model
# model = model_wrapper.model

# # ✅ Load weights into the internal model
# model.load_weights(checkpoint_path)
# print("✅ Loaded pretrained MesoInception4 weights successfully!")


✅ Loaded pretrained MesoInception4 weights successfully!


In [ ]:
# app = FaceAnalysis(name='buffalo_l')
# app.prepare(ctx_id=0, det_size=(224, 224))

# # -------------------------------
# # 4️⃣ Preprocess & predict
# # -------------------------------
# def preprocess_face(face_img):
#     face_img = cv2.resize(face_img, (256, 256))
#     face_img = face_img.astype("float32") / 255.0
#     face_img = img_to_array(face_img)
#     return np.expand_dims(face_img, axis=0)

# def detect_deepfake(image_path):
#     img = cv2.imread(image_path)
#     if img is None:
#         print(f"⚠️ Image not found: {image_path}")
#         return None

#     faces = app.get(img)
#     if not faces:
#         print(f"⚠️ No face detected in {image_path}")
#         return None

#     # pick largest face
#     face = max(faces, key=lambda x: (x.bbox[2]-x.bbox[0])*(x.bbox[3]-x.bbox[1]))
#     x1, y1, x2, y2 = map(int, face.bbox)
#     face_crop = img[y1:y2, x1:x2]

#     if face_crop.size == 0:
#         print(f"⚠️ Empty crop for {image_path}")
#         return None

#     face_input = preprocess_face(face_crop)
#     pred = model.predict(face_input)[0][0]
#     return {"real_prob": 1 - pred, "fake_prob": pred}

# # -------------------------------
# # 5️⃣ Example
# # -------------------------------
# image_path = "/content/r2.jpeg"
# result = detect_deepfake(image_path)
# if result:
#     print(f"Real: {result['real_prob']:.3f}, Fake: {result['fake_prob']:.3f}")


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (224, 224)
1/1 ━━━━━━━━

In [ ]:
# # -------------------------------
# # 1️⃣ Install required packages
# # -------------------------------

# # -------------------------------
# # 2️⃣ Imports
# # -------------------------------
# import cv2
# import numpy as np
# from insightface.app import FaceAnalysis
# from tensorflow.keras.models import load_model
# from tensorflow.keras.preprocessing.image import img_to_array
# from tqdm import tqdm

# # -------------------------------
# # 3️⃣ Initialize InsightFace for face detection
# # -------------------------------
# app = FaceAnalysis(name='buffalo_l')
# app.prepare(ctx_id=0, det_size=(224, 224))

# # -------------------------------
# # 4️⃣ Load MesoInception4 (.h5) model
# # -------------------------------
# # checkpoint_path = "/content/MesoNet/weights/MesoInception_DF.h5"
# checkpoint_path = "/content/MesoNet/weights/MesoInception_DF.h5"
# deepfake_model = load_model(checkpoint_path)

# # -------------------------------
# # 5️⃣ Preprocess face crop
# # -------------------------------
# def preprocess_face(face_img):
#     face_img = cv2.resize(face_img, (224, 224))
#     face_img = face_img.astype("float") / 255.0
#     face_img = img_to_array(face_img)
#     face_img = np.expand_dims(face_img, axis=0)
#     return face_img

# # -------------------------------
# # 6️⃣ Deepfake detection function
# # -------------------------------
# def detect_deepfake(image_path):
#     img = cv2.imread(image_path)
#     if img is None:
#         print(f"⚠️ Image not found: {image_path}")
#         return None

#     faces = app.get(img)
#     if not faces:
#         print(f"⚠️ No face detected in {image_path}")
#         return None

#     # Take the largest face
#     face = max(faces, key=lambda x: (x.bbox[2]-x.bbox[0])*(x.bbox[3]-x.bbox[1]))
#     x1, y1, x2, y2 = map(int, face.bbox)
#     face_crop = img[y1:y2, x1:x2]

#     if face_crop.size == 0:
#         print(f"⚠️ Empty face crop for {image_path}")
#         return None

#     # Preprocess
#     face_input = preprocess_face(face_crop)

#     # Predict
#     pred = deepfake_model.predict(face_input)[0][0]  # Assuming single output (sigmoid)
#     return {"real_prob": 1-pred, "fake_prob": pred}

# # -------------------------------
# # 7️⃣ Example usage
# # -------------------------------
# image_path = "/content/test_face.jpg"
# result = detect_deepfake(image_path)
# if result:
#     print(f"Real probability: {result['real_prob']:.3f}, Fake probability: {result['fake_prob']:.3f}")

# # -------------------------------
# # 8️⃣ Batch processing for CSV
# # -------------------------------


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (224, 224)


ValueError: No model config found in the file at /content/MesoNet/weights/MesoInception_DF.h5.

In [ ]:
# import pandas as pd

# csv_path = "/content/identity_pairs_val.csv"  # Your CSV
# df = pd.read_csv(csv_path)

# similarities = []  # To store similarity + deepfake info

# for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing CSV"):
#     img_path = row['img1']  # or img2
#     deepfake_result = detect_deepfake(img_path)
#     if deepfake_result:
#         similarities.append({
#             "img": img_path,
#             "label": row['label'],
#             "real_prob": deepfake_result["real_prob"],
#             "fake_prob": deepfake_result["fake_prob"]
#         })

# results_df = pd.DataFrame(similarities)
# results_df.to_csv("deepfake_results.csv", index=False)
# print("✅ Saved deepfake detection results to deepfake_results.csv")


In [ ]:
# import torch
# import torch.nn.functional as F
# from transformers import AutoImageProcessor, SiglipForImageClassification
# from PIL import Image
# import cv2
# from insightface.app import FaceAnalysis

# # ----------------------
# # Load deepfake model & processor
# # ----------------------
# model_name = "prithivMLmods/deepfake-detector-model-v1"
# model = SiglipForImageClassification.from_pretrained(model_name)
# processor = AutoImageProcessor.from_pretrained(model_name)
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model = model.to(device)
# model.eval()

# # ----------------------
# # Initialize InsightFace
# # ----------------------
# app = FaceAnalysis(name='buffalo_l')
# app.prepare(ctx_id=0, det_size=(224, 224))

# def detect_deepfake_v1(image_path):
#     # read image
#     img = cv2.imread(image_path)
#     if img is None:
#         return None
#     faces = app.get(img)
#     if not faces:
#         return None

#     # choose the largest face
#     face = max(faces, key=lambda x: (x.bbox[2]-x.bbox[0])*(x.bbox[3]-x.bbox[1]))
#     x1, y1, x2, y2 = map(int, face.bbox)
#     face_crop = img[y1:y2, x1:x2]
#     if face_crop.size == 0:
#         return None

#     # Convert crop to PIL (RGB)
#     face_pil = Image.fromarray(cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB))
#     inputs = processor(images=face_pil, return_tensors="pt").to(device)
#     with torch.no_grad():
#         outputs = model(**inputs)
#         probs = F.softmax(outputs.logits, dim=1).squeeze().tolist()
#     # assuming id2label mapping: "0": fake, "1": real
#     real_prob = probs[1]
#     fake_prob = probs[0]
#     return {"real_prob": real_prob, "fake_prob": fake_prob}

# # ----------------------
# # Test
# # ----------------------
# # res = detect_deepfake_v1("/content/r1.jpeg")
# # if res:
# #     print("Real:", res["real_prob"], "Fake:", res["fake_prob"])


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (224, 224)
Real: 0.9858

In [ ]:
# res = detect_deepfake_v1("/content/f5.jpeg")
# if res:
#     print("Real:", res["real_prob"], "Fake:", res["fake_prob"])


Real: 0.8994782567024231 Fake: 0.10052172094583511


In [ ]:
# import torch
# import torch.nn.functional as F
# from transformers import AutoImageProcessor, SiglipForImageClassification
# from PIL import Image
# import cv2
# from insightface.app import FaceAnalysis
# import pandas as pd
# from tqdm import tqdm

# # ----------------------------------------------------------
# # 1️⃣ Device setup
# # ----------------------------------------------------------
# device = "cuda" if torch.cuda.is_available() else "cpu"

# # ----------------------------------------------------------
# # 2️⃣ Load model and processor
# # ----------------------------------------------------------
# model_name = "prithivMLmods/deepfake-detector-model-v1"
# model = SiglipForImageClassification.from_pretrained(model_name).to(device)
# processor = AutoImageProcessor.from_pretrained(model_name)
# model.eval()

# # ----------------------------------------------------------
# # 3️⃣ Initialize Face Detector (InsightFace)
# # ----------------------------------------------------------
# app = FaceAnalysis(name='buffalo_l')
# app.prepare(ctx_id=0, det_size=(224, 224))

# # ----------------------------------------------------------
# # 4️⃣ Function: detect face & predict deepfake probability
# # ----------------------------------------------------------
# def predict_deepfake(img_path):
#     try:
#         img = cv2.imread(img_path)
#         if img is None:
#             return 0.5, 0.5  # fallback neutral probability

#         faces = app.get(img)

#         if faces:
#             # pick largest face
#             face = max(faces, key=lambda x: (x.bbox[2]-x.bbox[0])*(x.bbox[3]-x.bbox[1]))
#             x1, y1, x2, y2 = map(int, face.bbox)
#             face_crop = img[y1:y2, x1:x2]
#         else:
#             # fallback: use full image
#             face_crop = img

#         # ensure non-empty
#         if face_crop.size == 0:
#             face_crop = img

#         # convert to PIL RGB and resize
#         face_pil = Image.fromarray(cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB)).resize((224, 224))

#         # prepare input
#         inputs = {k: v.to(device) for k, v in processor(images=face_pil, return_tensors="pt").items()}

#         with torch.no_grad():
#             outputs = model(**inputs)
#             probs = F.softmax(outputs.logits, dim=1).squeeze().cpu().tolist()

#         # model labels: 0=fake, 1=real
#         real_prob, fake_prob = probs[1], probs[0]
#         return real_prob, fake_prob

#     except Exception as e:
#         print(f"⚠️ Error processing {img_path}: {e}")
#         return 0.5, 0.5


# # ----------------------------------------------------------
# # 5️⃣ Evaluate all images from CSV
# # ----------------------------------------------------------
# def evaluate_deepfake_csv(csv_path, save_csv=True):
#     df = pd.read_csv(csv_path)
#     real_probs, fake_probs = [], []

#     print(f"🧠 Evaluating {len(df)} images...")
#     for path in tqdm(df["image_path"]):
#         real_p, fake_p = predict_deepfake(path)
#         real_probs.append(real_p)
#         fake_probs.append(fake_p)

#     df["pred_real_prob"] = real_probs
#     df["pred_fake_prob"] = fake_probs

#     # average probabilities grouped by ground truth
#     avg_stats = (
#         df.groupby("label")[["pred_real_prob", "pred_fake_prob"]]
#         .mean()
#         .rename_axis("ground_truth")
#     )

#     print("\n📊 Average predicted probabilities:")
#     print(avg_stats)

#     if save_csv:
#         out_path = csv_path.replace(".csv", "_results.csv")
#         df.to_csv(out_path, index=False)
#         print(f"\n✅ Saved detailed results to: {out_path}")

#     return df, avg_stats

# # ----------------------------------------------------------
# # 6️⃣ Run evaluation
# # ----------------------------------------------------------
# csv_path = "/content/forgery_val.csv"  # update path to your CSV
# results_df, avg_probs = evaluate_deepfake_csv(csv_path)


/usr/local/lib/python3.12/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:123: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (224, 224)
🧠 Evaluating

100%|██████████| 4000/4000 [00:00<00:00, 19150.91it/s]


📊 Average predicted probabilities:
              pred_real_prob  pred_fake_prob
ground_truth                                
fake                     0.5             0.5
real                     0.5             0.5

✅ Saved detailed results to: /content/forgery_val_results.csv


In [ ]:
# r, f = predict_deepfake("/content/f5.jpeg")
# print(f"Real: {r:.3f}, Fake: {f:.3f}")


Real: 0.899, Fake: 0.101


DisabledFunctionError: cv2.imshow() is disabled in Colab, because it causes Jupyter sessions
to crash; see https://github.com/jupyter/notebook/issues/3935.
As a substitution, consider using
  from google.colab.patches import cv2_imshow


Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/Sentinel_FaceV1/Forgery_Dataset/fake/7SN7UB7ZFG.jpg  
  inflating: /content/drive/MyDrive/Sentinel_FaceV1/Forgery_Dataset/fake/J5EZH7VQ5R.jpg  
  inflating: /content/drive/MyDrive/Sentinel_FaceV1/Forgery_Dataset/fake/AX5R5WK2BJ.jpg  
  inflating: /content/drive/MyDrive/Sentinel_FaceV1/Forgery_Dataset/fake/IMZTDGOZ51.jpg  
  inflating: /content/drive/MyDrive/Sentinel_FaceV1/Forgery_Dataset/fake/8GK1GOH3DE.jpg  
  inflating: /content/drive/MyDrive/Sentinel_FaceV1/Forgery_Dataset/fake/77QV1G37QU.jpg  
  inflating: /content/drive/MyDrive/Sentinel_FaceV1/Forgery_Dataset/fake/GOQH4D1Y1U.jpg  
  inflating: /content/drive/MyDrive/Sentinel_FaceV1/Forgery_Dataset/fake/FE5TTE3NP6.jpg  
  inflating: /content/drive/MyDrive/Sentinel_FaceV1/Forgery_Dataset/fake/PEEPEID21U.jpg  
  inflating: /content/drive/MyDrive/Sentinel_FaceV1/Forgery_Dataset/fake/4EOJ1N355X.jpg  
  inflating: /content/drive/MyDrive/Sentinel_Face

In [ ]:
# # -------------------------------
# # 1️⃣ Imports
# # -------------------------------
# import os
# import cv2
# import torch
# import torch.nn.functional as F
# import pandas as pd
# from PIL import Image
# from tqdm import tqdm

# from insightface.app import FaceAnalysis
# from torchvision import transforms
# from torchvision.models import xception
# # If you have the FaceForensics++ Xception weights, we’ll load them later

# # -------------------------------
# # 2️⃣ Device
# # -------------------------------
# device = "cuda" if torch.cuda.is_available() else "cpu"

# # -------------------------------
# # 3️⃣ Initialize InsightFace
# # -------------------------------
# app = FaceAnalysis(name='buffalo_l')  # Lightweight & accurate model
# app.prepare(ctx_id=0 if device=="cuda" else -1, det_size=(224,224))

# # -------------------------------
# # 4️⃣ Load Xception pretrained on FaceForensics++
# # -------------------------------
# # You need the checkpoint from FaceForensics++
# # Example path: "/content/Xception_FF++_weights.pth"
# xception_model = xception(pretrained=False, num_classes=2)  # 2 classes: fake / real
# # checkpoint_path = "/content/Xception_FF++_weights.pth"
#     # <-- update this path
# checkpoint_path="/content/xception-b5690688.pth"

# # Load weights
# checkpoint = torch.load(checkpoint_path, map_location=device)
# xception_model.load_state_dict(checkpoint)
# xception_model = xception_model.to(device)
# xception_model.eval()

# # -------------------------------
# # 5️⃣ Preprocessing
# # -------------------------------
# preprocess = transforms.Compose([
#     transforms.Resize((299,299)),
#     transforms.ToTensor(),
#     transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
# ])

# # -------------------------------
# # 6️⃣ Prediction function
# # -------------------------------
# def predict_deepfake(img_path):
#     try:
#         img = cv2.imread(img_path)
#         if img is None:
#             return None, None

#         faces = app.get(img)
#         if not faces:
#             return None, None

#         # pick largest face
#         face = max(faces, key=lambda x: (x.bbox[2]-x.bbox[0])*(x.bbox[3]-x.bbox[1]))
#         x1, y1, x2, y2 = map(int, face.bbox)
#         face_crop = img[y1:y2, x1:x2]
#         if face_crop.size == 0:
#             return None, None

#         # PIL + preprocessing
#         face_pil = Image.fromarray(cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB))
#         input_tensor = preprocess(face_pil).unsqueeze(0).to(device)

#         # inference
#         with torch.no_grad():
#             logits = xception_model(input_tensor)
#             probs = F.softmax(logits, dim=1).squeeze().tolist()

#         # id2label: 0=fake, 1=real
#         real_prob, fake_prob = probs[1], probs[0]
#         return real_prob, fake_prob

#     except Exception as e:
#         print(f"⚠️ Error processing {img_path}: {e}")
#         return None, None

# # -------------------------------
# # 7️⃣ Evaluate CSV
# # -------------------------------
# def evaluate_deepfake_csv(csv_path, save_csv=True):
#     df = pd.read_csv(csv_path)
#     real_probs, fake_probs = [], []

#     print(f"🧠 Evaluating {len(df)} images...")
#     for path in tqdm(df["image_path"]):
#         real_p, fake_p = predict_deepfake(path)
#         real_probs.append(real_p)
#         fake_probs.append(fake_p)

#     df["pred_real_prob"] = real_probs
#     df["pred_fake_prob"] = fake_probs

#     # average probabilities grouped by ground truth
#     avg_stats = (
#         df.groupby("label")[["pred_real_prob", "pred_fake_prob"]]
#         .mean()
#         .rename_axis("ground_truth")
#     )

#     print("\n📊 Average predicted probabilities:")
#     print(avg_stats)

#     if save_csv:
#         out_path = csv_path.replace(".csv", "_results.csv")
#         df.to_csv(out_path, index=False)
#         print(f"\n✅ Saved detailed results to: {out_path}")

#     return df, avg_stats

# # -------------------------------
# # 8️⃣ Run evaluation
# # -------------------------------
# csv_path = "/content/forgery_val.csv"  # <-- update path
# results_df, avg_probs = evaluate_deepfake_csv(csv_path)


ImportError: cannot import name 'xception' from 'torchvision.models' (/usr/local/lib/python3.12/dist-packages/torchvision/models/__init__.py)

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/Sentinel_FaceV1/content/processed_faces/real/35249.jpg  
  inflating: /content/drive/MyDrive/Sentinel_FaceV1/content/processed_faces/real/50116.jpg  
  inflating: /content/drive/MyDrive/Sentinel_FaceV1/content/processed_faces/real/55046.jpg  
  inflating: /content/drive/MyDrive/Sentinel_FaceV1/content/processed_faces/real/62951.jpg  
  inflating: /content/drive/MyDrive/Sentinel_FaceV1/content/processed_faces/real/09486.jpg  
  inflating: /content/drive/MyDrive/Sentinel_FaceV1/content/processed_faces/real/11732.jpg  
  inflating: /content/drive/MyDrive/Sentinel_FaceV1/content/processed_faces/real/25664.jpg  
  inflating: /content/drive/MyDrive/Sentinel_FaceV1/content/processed_faces/real/14668.jpg  
  inflating: /content/drive/MyDrive/Sentinel_FaceV1/content/processed_faces/real/49412.jpg  
  inflating: /content/drive/MyDrive/Sentinel_FaceV1/content/processed_faces/real/40549.jpg  
  inflating: /conte

Cloning into 'FaceForensics'...
remote: Enumerating objects: 414, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 414 (delta 29), reused 14 (delta 14), pack-reused 368 (from 1)
Receiving objects: 100% (414/414), 59.81 MiB | 14.99 MiB/s, done.
Resolving deltas: 100% (166/166), done.


ModuleNotFoundError: No module named 'xception'

dataset  detect_from_video.py  network	README.md  requirements.txt


Trying DFDT

/usr/local/lib/python3.12/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:123: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (224, 224)
🧠 Evaluating

100%|██████████| 4000/4000 [00:00<00:00, 16098.73it/s]



📊 Average predicted probabilities:
             pred_real_prob pred_fake_prob
ground_truth                              
fake                    NaN            NaN
real                    NaN            NaN

✅ Saved detailed results to: /content/forgery_val_results.csv


generateing csv files

✅ Train CSV saved: forgery_train.csv (4897 samples)
✅ Validation CSV saved: forgery_val.csv (1225 samples)
